In [22]:
# import necessary libraries
import os, csv
import shutil

In [2]:
#setting directories
micro_sample_cvs = "/Users/anhnguyen/Desktop/research/scraping_Python/micro-sample_Feb17.csv"
wget_folder = "/Users/anhnguyen/Desktop/research/scraping_Python/wget_accept"
no_dir_folder = "/Users/anhnguyen/Desktop/research/scraping_Python/no_dir"
learning_wget = "/Users/anhnguyen/Desktop/research/scraping_Python/learning_wget"

In [3]:
sample = [] # make empty list
with open(micro_sample_cvs, 'r', encoding = 'Windows-1252')\
as csvfile: # open file; the windows-1252 encoding looks weird but works for this
    reader = csv.DictReader(csvfile) # create a reader
    for row in reader: # loop through rows
        sample.append(row) # append each row to the list
        
#note: each row, sample[i] is a dictionary with keys as column name and value as info

In [10]:
# turning this into tuples we can use with wget!
# first, make some empty lists
url_list = []
name_list = []
terms_list = []

# now let's fill these lists with content from the sample
for school in sample:
    url_list.append(school["URL"])
    name_list.append(school["SCHNAM"])
    terms_list.append(school["ADDRESS"])

In [11]:
tuple_list = list(zip(url_list, name_list))
# Let's check what these tuples look like:
print(tuple_list[:3])
print("\n", tuple_list[1][1].title())

[('https://www.richland2.org/charterhigh/', 'RICHLAND TWO CHARTER HIGH'), ('https://www.polk.edu/lakeland-gateway-to-college-high-school/', 'POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL'), ('https://www.nhaschools.com/schools/rivercity/Pages/default.aspx', 'RIVER CITY SCHOLARS CHARTER ACADEMY')]

 Polk State College Collegiate High School


### Helper Functions

In [51]:
def format_folder_name (k, name):
    """Format a folder nicely for easy access"""
    if k < 10: # Add two zeros to the folder name if k is less than 10 (for ease of organizing the output folders)
        dirname = "00" + str(k) + " " + name
    elif k < 100: # Add one zero if k is less than 100
        dirname = "0" + str(k) + " " + name
    else: # Add nothing if k>100
        dirname = str(k) + " " + name
    return dirname

def run_wget_command(link, parent_folder, my_folder):
    """wget on link and print output to appropriate folders"""
    #navigate to parent folder
    os.chdir(parent_folder)
    # create dir my_folder if it doesn't exist yet
    if not os.path.exists(my_folder):
        os.makedirs(my_folder)
    #navigate to the correct folder, ready to wget
    os.chdir(my_folder)
    os.system('wget -np --no-parent --show-progress --progress=dot --recursive --level=3 --convert-links --retry-connrefused \
         --random-wait --no-cookies --secure-protocol=auto --no-check-certificate --execute robots=off \
         --user-agent="Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:11.0) Gecko/20100101 Firefox/11.0" \
          --accept .html' + ' ' + link)
    

def contains_html(my_folder):
    """check if a wget is success by checking if a directory has a html file"""

    #did wget get anything?
#     os.chdir(my_folder)
#     list_of_files = os.listdir(".")
#     root = "/home/patate/directory/"
#     path = os.path.join(root, "targetdirectory")

    for r,d,f in os.walk(my_folder):
        for file in f:
            if file.endswith('.html'):
                return True
    return False  
#     #is there a html file
#     if (True in [file_name.endswith('.html') for file_name in list_of_files]):
#         return True
#     return False

def write_to_file(num, link, file_name):
    with open(file_name, "a") as text_file:
        text_file.write(str(num) + "\t" + link +"\n")
        
def reset(folder, text_file_1, text_file_2):
    """Deletes all files in a folder and set 2 text files to blank"""
    parent_folder = folder[: folder.rindex('/')]
    shutil.rmtree(folder)
    os.makedirs(folder)
    filelist = [ f for f in os.listdir(folder) if f.endswith(".bak") ]
    for f in filelist:
        os.unlink(f)
    for file_name in [text_file_1, text_file_2]:
        if os.path.exists(file_name):
            with open(file_name, "w") as text_file:
                text_file.write("")

In [9]:
#testing methods
print(format_folder_name(30, "name me"))



'030 name me'

### Running wget

In [17]:
# set up file directories
success_file = "/Users/anhnguyen/Desktop/research/scraping_Python/success.txt"
fail_file = "/Users/anhnguyen/Desktop/research/scraping_Python/fail.txt"

In [52]:
reset(wget_folder, success_file, fail_file)

In [53]:

k=0 # initialize this numerical variable k, which keeps track of which entry in the sample we are on.

#testing the first 10 tuples
tuple_test = tuple_list[0:5]

for tup in tuple_test:
    school_title = tup[1].title()
    k += 1 # Add one to k, so we start with 1 and increase by 1 all the way up to entry # 300
    print("Capturing website data for", school_title + ", which is school #" + str(k), "of 300...")
    
    # use the tuple to create a name for the folder
    dirname = format_folder_name(k, school_title)
    
    run_wget_command(tup[0], wget_folder, dirname)
    
    school_folder = wget_folder + '/'+ dirname
    if contains_html(school_folder):
        write_to_file(k, tup[0], success_file )
    else :
        write_to_file(k, tup[0], fail_file)
    

Capturing website data for Richland Two Charter High, which is school #1 of 300...
Capturing website data for Polk State College Collegiate High School, which is school #2 of 300...
Capturing website data for River City Scholars Charter Academy, which is school #3 of 300...
Capturing website data for Detroit Enterprise Academy, which is school #4 of 300...
Capturing website data for Lighthouse Community Sch Inc, which is school #5 of 300...
